In [ ]:
# %matplotlib 
import json
import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import urllib, json

In [ ]:
API_KEY = ''
meetup_group = 'central_london_data_science'
max_event_count = 100
max_per_page = 200

In [ ]:
def get_json_from_meetup_api(url):
    full_url = url + '&key=' + API_KEY
    response = urllib.urlopen(full_url)
    return json.loads(response.read())

def get_members(group_name, offset = 0):
    print group_name, offset
    url = ('https://api.meetup.com/2/members?&sign=true&photo-host=public&group_urlname='
           +group_name
           +'&page='
           +str(max_per_page)
           +'&offset='
           +str(offset))
    return get_json_from_meetup_api(url)

def get_events(group_name):
    url = ('https://api.meetup.com/2/events?&sign=true&photo-host=public&group_urlname='
           +group_name
           +'&status=past,upcoming&page='
           +str(max_event_count))
    return get_json_from_meetup_api(url)

def get_group_info(group_name):
    url = 'https://api.meetup.com/'+group_name+'?&sign=true&photo-host=public'
    return get_json_from_meetup_api(url)

def get_all_members(group_name, count):
    loops = count/max_per_page
    extra_loop = count%max_per_page is not 0
    tota_loops = loops + extra_loop
    print (tota_loops +' request(s) being made')
    store_members = []
    for i in range(tota_loops):
        current_get = get_members(group_name, offset = i)
        store_members += current_get['results']
    return store_members

def get_epoc_in_days_since(start_epoc, end_epoc):
    start = datetime.datetime.fromtimestamp(start_epoc/1000)
    end = datetime.datetime.fromtimestamp(end_epoc/1000)
    return (end - start).days

In [ ]:
group_info = get_group_info(meetup_group)

In [ ]:
group_start =  datetime.datetime.fromtimestamp(group_info['created']/1000)
group_start

In [ ]:
group_age = (datetime.datetime.now() -  group_start).days
group_age

In [ ]:
group_info['members']

In [ ]:
all_member_data = get_all_members(meetup_group, group_info['members'])

In [ ]:
len(all_member_data)

In [ ]:
all_member_data[0]['joined']

In [ ]:
group_info['members']

In [ ]:
# get date joined in days resolution
dateJoined = [x["joined"]/1000 for x in all_member_data if 'joined' in x.keys()]

In [ ]:
len(dateJoined)

In [ ]:
epochs = sorted(dateJoined)
when_people_joined = [datetime.datetime.fromtimestamp(x) for x in epochs]

In [ ]:
len(sorted(when_people_joined))

In [ ]:
running_total_of_members = []
for i in range(group_age):
    sum_to_day = len([j for j in when_people_joined if (datetime.datetime.now()-j).days <= i])
    running_total_of_members.append(sum_to_day)

In [ ]:
eventData = get_events(meetup_group)['results']

In [ ]:
event_count = len(eventData)
event_count

In [ ]:
create_event_date = [datetime.datetime.fromtimestamp(x['created']/1000) for x in eventData]
start_event_date = [datetime.datetime.fromtimestamp(x['time']/1000) for x in eventData]

create_event_day_from_group_start = [(x-group_start).days  for x in create_event_date]
start_event_day_from_group_start = [(x-group_start).days  for x in start_event_date]

len(create_event_day_from_group_start),len(start_event_day_from_group_start)

In [ ]:
plt.plot(range(group_age),running_total_of_members,color='g')
for i in create_event_day_from_group_start:
    plt.axvline(x=i,color='r', linewidth=0.3)
for i in start_event_day_from_group_start:
    plt.axvline(x=i,color='b', linewidth=0.3)

In [ ]:
members = mpatches.Patch(color='g', label='No. of members')
event_anounce = mpatches.Patch(color='r', label='Event anounced')
event_start = mpatches.Patch(color='b', label='Event start')

plt.legend(handles=[members,event_anounce,event_start],loc=4)
plt.show()